#MNIST Para Principiantes

En esta libreta se sigue el tutorial de MNIST For ML Beginners presentado en el proyecto de TensorFlow.

Los objetivos de este tutorial es conocer que es MNIST ademas de la regresion softmax.

MNIST es un dataset de vision por computadora. Consiste en imagenes de digitos escritos a mano tambien incluye etiquetas de cada imagen donde el nombre de cada etiqueta es el digito que se presenta en la imagen.

Ademas en este tutorial se ve un modelo de entrenamiento que revisa las imagenes y predice que digitos hay en ellas dicho modelo se llama regresion softmax.


#Los Datos de MNIST

Los datos de MNIST se descargan mediante el archivo que se encuentra en el proyecto TensorFlow en github llamado input_data.py
este archivo descarga los datos que se tratan en tres partes diferentes las cuales son:

* mnist.train (Contiene 55,00 puntos de datos de entrenamiento)
* mnist.test (Contiene 10.000 puntos de datos de prueba)
* mnist.validation (Contiene 5.000 puntos de datos de validación)

En machine learning es importante tener los datos de forma separada para que con los datos que se aprenda se pueda generalizar.

Como se menciona arriba el dataset MNIST consta de dos partes una es la imagen del digito escrito a mano y la otra la etiqueta con el nombre del correspondiente digito en la imagen a estos dos valores se les llamara "xs" y "ys" donde xs representa las imagenes y ys las etiquetas. Los datos de mnist.train y mnist.test contienen los valores de xs y ys.

Cada imagen es de tamano de 28 x 28 pixeles lo cual se puede interpretar como un vector con 784 numeros.

El resultado es que mnist.train.images es un tensor (Un arreglo de dimension n) con el siguiente tamano [55000, 784]. El primer indice corresponde al indice de las imagenes y el segundo indice a el tamano en pixeles que tiene cada imagen. 

Cada posicion en el tensor representa un pixel que se encuentra en una imagen cada pixel tiene un valor en el rango de 0 a 1.

Las etiquetas en MNIST estan en el rango de 0-9 los cuales son los digitos que pueden contener estas imagenes. Para utilizar las etiquetas se utiliza "one-hot vectors" lo cual es un vector que en la mayoria de sus dimensiones contiene un 0 y en una dimension tiene un valor unico en este caso 1. 

Por ejemplo si se tiene la etiqueta 3 se representa por el siguiente vector [0, 0, 0, 1, 0, 0, 0, 0, 0].

A continuacion se muestra el codigo para descargar los datos de MNIST. 


In [25]:
import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#Regresión Softmax

Se sabe que en MNIST cada imagen representa un digito el cual esta en el rango de 0-9. La idea de utilizar un modelo de entrenamiento que por ejemplo dada una imagen de MNIST que contenga por decir un 9 se pueda decir con cierta probabilidad que digito le corresponde a dicha imagen se espera que el modelo de una probabilidad muy alta de que sea el valor 9 pero tambien se puede esperar una probabilidad pequena de que sea un 8 y una mas pequena probabilidad de que sea otro digito.

Este es un caso clasico de una regresion softmax en donde se busca asignar probabilidades a un objeto o a diferentes cosas.

Una regresion softmax consiste de dos pasos:

1) Se agrega la evidencia que se tiene en ciertas clases.
2) Se convierte dicha evidencia en probabilidades.

Para determinar si una imagen pertenece a una clase en especial se utiliza una suma ponderada en donde se cuentan la intensidad de los pixeles si la suma ponderada da un valor negativo para una clase en particular significa que la imagen no pertenece a esa clase y si da positivo significa que puede pertenecer a esa clase.

Ademas se necesita una evidencia extra a la cual se le llama bias. Basicamanete se quiere decir que algunas cosas son independientes de la entrada. El resultado es la evidencia para una clase i dada una entrada x es:

$$\text{evidence}_i = \sum_j W_{i,~ j} x_j + b_i$$

donde wi son los pesos y bi es el bias para la clase i, ademas j es el indice para la suma de los pixeles en la entrada de la imagen x.

Se convierte la evidencia en probabilidades predecidas por la regresion softmax:

$$y = \text{softmax}(\text{evidence})$$

Aqui la regresion softmax sirve como una funcion de activacion dando forma a la salida de la funcion lineal a utilizar en la forma que se desea en este caso una distribucion de probabilidad con 10 casos.

La funcion softmax se define como:

$$\text{softmax}(x) = \text{normalize}(\exp(x))$$

Si se expande la ecuacion se obtiene:

$$\text{softmax}(x)_i = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

La exponenciación significa que una unidad mas de evidencia incrementa el peso para cualquier hipotesis caso contrario una unidad menos de evidencia decrementa el peso para cualquier hipotesis. Ninguna hipotesis tendra el valor 0 o un valor negativo. La regresion softmax normaliza los pesos gracias a esto se tiene una distribucion valida de probabilidad.

La regresion softmax se puede ver de la siguiente forma: 

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/softmax-regression-scalargraph.png)

Las salidas de la regresion softmax se pueden escribir de la siguiente manera:

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/softmax-regression-scalarequation.png)

Tambien se puede vectorizar el procedimiento anterior transformandolo en una multiplicacion de matrices y la suma de un vector como se ve a continuacion:


![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/softmax-regression-vectorequation.png)

Se puede escribir mas compacto como por ejemplo:

$$y = \text{softmax}(Wx + b)$$




#Implementando la regresion

Se utiliza TensorFlow el cual utiliza operaciones interactivas que se ejecutan fuera de python a continuacion se importa la libreria:

In [26]:
import tensorflow as tf

Se describen estas operaciones manipulando variables simbolicas se procede a crear una:

In [27]:
x = tf.placeholder(tf.float32, [None, 784])

x no es un valor en especifico. Es un placeholder, un valor que se pondra cuando se le pida a TensorFlow ejecutar alguna instruccion.

Tambien se necesitan los pesos y el bias para el modelo de regresion softmax.

In [28]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

A este tipo de variables se les da un valor inicial en este caso se inicializan ambos tensores w y b con valores de 0. Se procede aprender W y b por lo que no importa mucho con que valores se inicializen.

Como nota el tamano de W es [784, 10] porque se quiere multiplicar los vectores de 784 dimensiones para podrucir 10 vectores de pruebas para las clases de diferencia.

Ahora se puede implementar el modelo el cual solo toma una linea!

In [29]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Primero, se multiplica x por W con la expresion tf.matmul(x, W). Despues se suma b el cual es el bias y finalmente se aplica tf.nn.softmax.

#Entrenamiento

Primer para entrenar el modelo se necesita definir que significa un buen modelo. En machine learning actualmente se define que significa un mal modelo para esto se consideran los aspectos siguientes como el costo y la perdida y se trata de minimizar ambos aspectos. Los dos son queivalentes.

Una de las funciones mas comunes para costo es "cross-entropy" la cual se usa para codigos de comprension de informacion en la teoria de informacion pero esta idea se utiliza en muchas areas esta funcion se define a continuacion:

$$H_{y'}(y) = -\sum_i y'_i \log(y_i)$$

donde y es la prediccion de la distribucion de probabilidad y y' es la verdadera distribucion (El "one-hot vectors" que se dio en la entrada).

Para implementar cross-entropy se necesita primero agregar un nuevo placeholder pra la entrada de las respuestas correctas:

In [30]:
y_ = tf.placeholder(tf.float32, [None, 10])

Ahora se puede implementar cross-entropy $$-\sum y'\log(y)$$

In [31]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

Primero, tf.log calcula el logatitmo de cada elemento de y. Luego multiplica cada elemento de y_ con el correspondiente elemento de tf.log(y). Finalmente se suman todos los elementos al tensor con tf.reduce_sum
(Nota: Si se utiliza cross_entropy para un solo valor no da una descripcion muy buena por lo que se utiliza la suma de 100 cross-entropy esto da una mejor descripcion de que tan bueno es el modelo a utilizar.)

Ahora que se sabe que se desea del modelo es facil entrenarlo con TensorFlow ya que sabe el grafo completo de los calculos que se han hecho se usa automaticamente el algoritmo de backpropagation para determinar como afecta el costo de las variables que se utilizan y se pregunta como minimizar. Despues se aplica algun algoritmo de optimizacion para modificar las variables y reducir el costo.

In [32]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

En este caso se le pregunta a TensorFlow como minimizar cross_entropy usando el algoritmo de descenso de gradiente con una taza de aprendisaje de 0.01. El descenso de gradiente es un simple procedimiento donde TensorFlow simplemente cambia cada variable un poco de tal forma que se reduzca el costo. 

Lo que hace TensorFlow actualmente es regresar a escenas pasadas y agregar nuevas operaciones al grafo para implementar backpropagation y descenso de gradiente. Con esto se obtiene una sola operacion que cuando se ejecuta va a un paso en el algoritmo de descenso de gradiente y reduce su costo.

Se tiene ahora el modelo a utilizar pero para empezar se inicializan todas las variables que se crearon.


In [33]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

A continuacion se va entrenar con 1000 iteraciones:

In [34]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

En cada iteracion del loop se obtiene un batch de 100 puntos de datos aleatorios para el conjunto de entrenamiento. 
Se ejecuta train_step para dar feeding en los batches y reemplazar los placeholders.

Usar pequenos batches aleatorios de datos se le llama entrenamiento estocastico. En este caso el descenso de gradiente estocastico. Lo ideal seria utilizar toda la informacion que se tiene al entrnar ya que daria un mejor sentido a lo que se esta haciendo pero como es costoso mejor se utilizan diferentes subconjuntos cada vez esto es barato y se obtiene un beneficio parecido al otro.

#Evaluando el Modelo

Que tan bueno es el modelo?

Bueno primero se va a predecir la etiqueta correcta. tf.argmax es una funcion muy util la cual regresa el indice mas alto en la posicion de un tensor a lo largo de un eje. Por ejemplo tf.argmax(y, 1) es la etiqueta del modelo que peinsa que es el valor mas probable para cada posicion. mientras que tf.argmax(y_. 1) es la etiqueta correcta. Se puede utilizar tf.equal para revisar si la prediccion concuerda con la verdad.

In [35]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

La instruccion anterior regresa una lista de valores bool. Para determinar que fraccion de valores son correctos se aplica un cast con valores de punto flotante y luego se calcula la media.

Por ejemplo supongamos que obtenemos la siguiente lista [True, False, True, True] aplicando el cast se convierte en la siguiente lista [1, 0, 1, 1] y la media obtenida es 0.75.

Se muestra el codigo que realiza las operaciones dichas previamente:

In [36]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Finalmente se obtiene la exactidud al probar los datos:

In [37]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9154


El valor obtenido se debe de encontrar cerca del 91%

Esto es bueno? de entrada parece que si pero en realidad no. 